In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")

In [7]:
from springstone.data import get_data, get_missing_dates, create_train_test, create_df_for_prophet
from springstone.utils import bollinger_bands, moving_average, prophet_preprocessing, prophet_non_business_days
import pandas as pd
import numpy as np
from prophet import Prophet
from prophet.plot import plot_forecast_component_plotly
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [3]:
hist = get_data('AAPL')

[*********************100%***********************]  1 of 1 completed


In [4]:
hist

,Open,High,Low,Close,Volume
Date,,,,,
2012-02-21,18.102858,18.387501,18.004286,18.387501,605595200
2012-02-22,18.324286,18.410357,18.181070,18.322857,483302400
2012-02-23,18.395714,18.493929,18.196428,18.442499,568027600
2012-02-24,18.559643,18.674999,18.522858,18.657499,415072000
2012-02-27,18.618214,18.875000,18.438572,18.777143,547582000
...,...,...,...,...,...
2022-02-14,167.369995,169.580002,166.559998,168.880005,86185500
2022-02-15,170.970001,172.949997,170.250000,172.789993,64286300
2022-02-16,171.850006,173.339996,170.050003,172.550003,61177400


In [5]:
prophet_preprocessing(hist, 'Close')

,ds,y
0,2012-02-21,18.387501
1,2012-02-22,18.322857
2,2012-02-23,18.442499
3,2012-02-24,18.657499
4,2012-02-25,18.657499
...,...,...
3646,2022-02-14,168.880005
3647,2022-02-15,172.789993
3648,2022-02-16,172.550003
3649,2022-02-17,168.880005


In [8]:
prophet_non_business_days(hist)

,holiday,ds
0,non business day,2012-02-25
1,non business day,2012-02-26
2,non business day,2012-03-03
3,non business day,2012-03-04
4,non business day,2012-03-10
...,...,...
1128,non business day,2022-01-30
1129,non business day,2022-02-05
1130,non business day,2022-02-06
1131,non business day,2022-02-12


In [9]:
non_business_days = get_missing_dates(hist, True)
an_array = np.full((non_business_days.shape[0], 1), np.nan)
an_array.shape

(1133, 1)

In [10]:
concatenated = pd.concat([hist[['Close']], pd.DataFrame(an_array, index=non_business_days, columns=['Close'])])

In [11]:
concatenated = concatenated.sort_index()
concatenated.rename_axis(index='Date', inplace=True)

In [12]:
concatenated = concatenated.fillna(method='ffill')

In [13]:
concatenated

,Close
Date,
2012-02-21,18.387501
2012-02-22,18.322857
2012-02-23,18.442499
2012-02-24,18.657499
2012-02-25,18.657499
...,...
2022-02-14,168.880005
2022-02-15,172.789993
2022-02-16,172.550003


In [14]:
df_prophet = create_df_for_prophet(concatenated)
df_train, df_test = create_train_test(df_prophet)

In [15]:
df_train

,ds,y
0,2012-02-21,18.387501
1,2012-02-22,18.322857
2,2012-02-23,18.442499
3,2012-02-24,18.657499
4,2012-02-25,18.657499
...,...,...
2916,2020-02-15,81.237503
2917,2020-02-16,81.237503
2918,2020-02-17,81.237503
2919,2020-02-18,79.750000


In [16]:
df_test

,ds,y
2921,2020-02-20,80.074997
2922,2020-02-21,78.262497
2923,2020-02-22,78.262497
2924,2020-02-23,78.262497
2925,2020-02-24,74.544998
...,...,...
3646,2022-02-14,168.880005
3647,2022-02-15,172.789993
3648,2022-02-16,172.550003
3649,2022-02-17,168.880005


In [17]:
non_business_days_df = pd.DataFrame({'holiday': 'non business day',
                                  'ds': non_business_days})

In [18]:
non_business_days_df

,holiday,ds
0,non business day,2012-02-25
1,non business day,2012-02-26
2,non business day,2012-03-03
3,non business day,2012-03-04
4,non business day,2012-03-10
...,...,...
1128,non business day,2022-01-30
1129,non business day,2022-02-05
1130,non business day,2022-02-06
1131,non business day,2022-02-12


In [ ]:
m_prophet = Prophet(holidays=non_business_days_df)
m_prophet.fit(df_train)

In [ ]:
n_days_prediction=90
future = m_prophet.make_future_dataframe(periods=n_days_prediction)
future

In [ ]:
forecast = m_prophet.predict(future)

In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'non business day']]

In [ ]:
m_prophet.plot(forecast);

In [ ]:
m_prophet.plot_components(forecast);

In [ ]:
#plot_forecast_component_plotly(m_prophet, forecast, 'non business day')

In [ ]:
forecast = forecast[- n_days_prediction:][['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'non business day']]

In [ ]:
forecast

In [ ]:
df_test[:n_days_prediction]

In [ ]:
mae_prophet = mean_absolute_error(df_test[:n_days_prediction]['y'], forecast['yhat'])
mape_prophet = mean_absolute_percentage_error(df_test[:n_days_prediction]['y'], forecast['yhat'])
print(f'MAE: {mae_prophet}')
print(f'MAPE: {mape_prophet}')